# The First Stars: Formation under Cosmic Ray Feedback -- Phase Diagrams

This notebook generates all simulation results figures for my current paper.  All simulations run on stampede.tacc.utexas.edu

In [ ]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
#mpl.rc('font', size=20.)
#mpl.rc('font', family='serif')
#mpl.rc('text', usetex=True)
import seaborn as sns
import pyGadget

In [ ]:
sns.set_context('poster')
mpl.rc('text', usetex=True)

In [ ]:
simV = pyGadget.sim.Simulation('stampede/no_bkg',length='pc')
sim0 = pyGadget.sim.Simulation('stampede/cr00',length='pc')
sim1 = pyGadget.sim.Simulation('stampede/cr01',length='pc')
sim2 = pyGadget.sim.Simulation('stampede/cr02',length='pc')
sim3 = pyGadget.sim.Simulation('stampede/cr03',length='pc')
sim4 = pyGadget.sim.Simulation('stampede/cr04',length='pc')
sim5 = pyGadget.sim.Simulation('stampede/cr05',length='pc')
sims = [simV, sim0, sim1, sim2, sim3, sim4, sim5]

In [ ]:
simname = {'vanilla':r'$u_{\rm \tiny CR} = 0$',
        'cr00': r'$u_{\rm \tiny CR} = u_0$', 
        'cr01': r'$u_{\rm \tiny CR} = 10 u_0$', 
        'cr02': r'$u_{\rm \tiny CR} = 10^2 u_0$', 
        'cr03': r'$u_{\rm \tiny CR} = 10^3 u_0$', 
        'cr04': r'$u_{\rm \tiny CR} = 10^4 u_0$',
        'cr05': r'$u_{\rm \tiny CR} = 10^5 u_0$'}
simlist = ['vanilla', 'cr02', 'cr03', 'cr04', 'cr05']
nsnap = [355, 310, 327, 318, 249, 213, 176]

In [ ]:
snaps= {}
for name, n, sim in zip(simlist, nsnap, sims):
    snaps[name] = sim.load_snapshot(n, 'ndensity', 'temp', 'particleIDs', 'h2frac', 'HDfrac', 'electron_frac')

In [ ]:
snapdata = {}
for sim in simlist:
    snap = snaps[sim]
    snapdata[sim] = pd.DataFrame(np.column_stack((snap.gas.particleIDs, snap.gas.ndensity, snap.gas.temp, snap.gas.h2frac, snap.gas.HDfrac, snap.gas.electron_frac)),
                               index=snap.gas.particleIDs, columns=['ID', 'ndensity', 'temp', 'h2frac', 'HDfrac', 'efrac'])

In [ ]:
ars_temp = pd.read_csv(os.getenv("HOME")+'/data/literature_data/Stacy_Bromm2007/f2_nocr.csv', names=['n', 'temp'])
ars_temp_hiSF = pd.read_csv(os.getenv("HOME")+'/data/literature_data/Stacy_Bromm2007/f2_1e6_high.csv', names=['n', 'temp'])
ars_temp_loSF = pd.read_csv(os.getenv("HOME")+'/data/literature_data/Stacy_Bromm2007/f2_1e6_low.csv', names=['n', 'temp'])
ars_mbe = pd.read_csv(os.getenv("HOME")+'/data/literature_data/Stacy_Bromm2007/f6_nocr.csv', names=['n', 'Mbe'])
ars_mbe_hiSF = pd.read_csv(os.getenv("HOME")+'/data/literature_data/Stacy_Bromm2007/f6_1e6_high.csv', names=['n', 'Mbe'])
ars_mbe_loSF = pd.read_csv(os.getenv("HOME")+'/data/literature_data/Stacy_Bromm2007/f6_1e6_low.csv', names=['n', 'Mbe'])
for df in [ars_temp, ars_temp_hiSF, ars_temp_loSF]:
    df['logn'] = np.log10(df.n)
    df['logT'] = np.log10(df.temp)
for df in [ars_mbe, ars_mbe_hiSF, ars_mbe_loSF]:
    df['logn'] = np.log10(df.n)
    df['logM'] = np.log10(df.Mbe)   

In [ ]:
average = {}
bins = np.linspace(-4,12,100)
for sim in simlist:
    print sim
    df = snapdata[sim]
    df.sort('ndensity', inplace=True)
    df['logn'] = np.log10(df.ndensity)
    # calculate the Bonnor-Ebert mass
    df['Mbe'] = 700 * (df.temp/200)**1.5 / np.sqrt(df.ndensity/1e4)
    # bin the data
    gb = df.groupby(pd.cut(df.logn, bins))
    # take the average of all quantities for each bin
    avg = gb.mean()
    # Calculate total mass above a given density
    avg['Mn'] = gb['ID'].count()[::-1].cumsum()[::-1] * 0.014770775438579679
    avg['logT'] = np.log10(avg.temp)
    avg['loge'] = np.log10(avg.efrac)
    avg['logM'] = np.log10(avg.Mbe)
    average[sim] = avg


In [ ]:
first_quartile = {}
third_quartile = {}
for sim in simlist:
    print sim
    df = snapdata[sim]
    gb = df.groupby(pd.cut(df.logn, bins))
    q1 = gb.quantile(q=.25)
    first_quartile[sim] = q1
    q1['logT'] = np.log10(q1.temp)
    q1['loge'] = np.log10(q1.efrac)
    q1['logM'] = np.log10(q1.Mbe)
    q2 = gb.quantile(q=.75)
    q2['logT'] = np.log10(q2.temp)
    q2['loge'] = np.log10(q2.efrac)
    q2['logM'] = np.log10(q2.Mbe)
    third_quartile[sim] = q2

In [ ]:
current_palette = sns.color_palette()[::-1]
sns.palplot(current_palette)

In [ ]:
current_palette = sns.color_palette()
f,ax = plt.subplots()
df = average['vanilla']
ax.fill_between(df.logn, first_quartile['vanilla'].logT, third_quartile['vanilla'].logT, 
                edgecolor='none', facecolor='.25', alpha=0.3)
ax.plot(df.logn, df.logT, color='.25', label=simname['vanilla'])
for sim in simlist[1:]:
    df = average[sim]
    ax.plot(df.logn, df.logT, label=simname[sim])
ax.fill_between(df.logn, first_quartile[sim].logT, third_quartile[sim].logT,
                edgecolor='none', facecolor=current_palette[3], alpha=0.3)
ax.plot(ars_temp.logn, ars_temp.logT, color='.25', ls='--', label="one-zone model")
ax.plot(ars_temp_hiSF.logn, ars_temp_hiSF.logT, color='.25', ls='-.', label="one-zone, high SF")
ax.plot(ars_temp_loSF.logn, ars_temp_loSF.logT, color='.25', ls=':', label="one-zone, low SF")
ax.legend(loc=0)
ax.set_xlim(-1, 12)
ax.set_ylim(1.75,3)
ax.set_xlabel('Log n [cm$^{-3}$]')
ax.set_ylabel('Log Temperature [K]')
f.savefig('figures/phase_diagrams/stacy_temp.png', bbox_inches='tight')

In [ ]:
current_palette = sns.color_palette()
f,ax = plt.subplots()
df = average['vanilla']
ax.fill_between(df.logn, first_quartile['vanilla'].logM, third_quartile['vanilla'].logM, 
                edgecolor='none', facecolor='.25', alpha=0.3)
ax.plot(df.logn, df.logM, color='.25', label=simname['vanilla'])
for sim in simlist[1:]:
    df = average[sim]
    ax.plot(df.logn, df.logM, label=simname[sim])
ax.fill_between(df.logn, first_quartile[sim].logM, third_quartile[sim].logM,
                edgecolor='none', facecolor=current_palette[3], alpha=0.3)
ax.plot(ars_mbe.logn, ars_mbe.logM, color='.25', ls='--', label="one-zone model")
ax.plot(ars_mbe_hiSF.logn, ars_mbe_hiSF.logM, color='.25', ls='-.', label="one-zone, high SF")
ax.plot(ars_mbe_loSF.logn, ars_mbe_loSF.logM, color='.25', ls=':', label="one-zone, low SF")
ax.legend(loc=0)
ax.set_xlim(-1, 12)
ax.set_ylim(-.25,6)
ax.set_xlabel('Log n [cm$^{-3}$]')
ax.set_ylabel('Log M$_{BE}$ [M$_{\odot}$]')
f.savefig('figures/phase_diagrams/stacy_Mbe.png', bbox_inches='tight')

In [ ]:
thick=4; thin=2
current_palette = sns.color_palette()
with sns.color_palette(n_colors=4):
    f,ax = plt.subplots()
df = average['vanilla']
ax.fill_between(df.ndensity, first_quartile['vanilla'].Mbe, third_quartile['vanilla'].Mbe, 
                edgecolor='none', facecolor='.25', alpha=0.2)
ax.plot(df.ndensity, df.Mbe, color='.25', label=simname['vanilla'], ls='-', lw=thick)
for sim in simlist[1:]:
    df = average[sim]
    ax.plot(df.ndensity, df.Mbe, label=simname[sim], ls='-', lw=thick)
df = average['vanilla']
ax.plot(df.ndensity, df.Mn, color='.25', ls='--', lw=thin)
for sim in simlist[1:]:
    df = average[sim]
    ax.plot(df.ndensity, df.Mn, ls='--', lw=thin)
ax.fill_between(df.ndensity, first_quartile[sim].Mbe, third_quartile[sim].Mbe,
                edgecolor='none', facecolor=current_palette[3], alpha=0.2)
ax.legend(loc=0)
ax.set_xlim(1e3, 1e4)
ax.set_ylim(7.5e2,2e3)
#ax.set_xscale('log')
#ax.set_yscale('log')
ax.set_xlabel('Log n [cm$^{-3}$]')
ax.set_ylabel('Log M$_{BE}$ [M$_{\odot}$]')
f.savefig('figures/phase_diagrams/Menc_Mbe.png', bbox_inches='tight')

In [ ]:
current_palette = sns.color_palette()[::-1]
f,ax = plt.subplots()
df = average['vanilla']
#ax.fill_between(df.logn, first_quartile['vanilla'].loge, third_quartile['vanilla'].loge, 
#                edgecolor='none', facecolor='.25', alpha=0.3)
ax.plot(df.logn, df.loge, color='.25', label=simname['vanilla'])
for sim in simlist[1:]:
    df = average[sim]
#    ax.fill_between(df.logn, first_quartile[sim].loge, third_quartile[sim].loge,
#                    edgecolor='none', facecolor=current_palette.pop(), alpha=0.3)
    ax.plot(df.logn, df.loge, label=simname[sim])
ax.legend(loc=0)
ax.set_xlim(8,12)
ax.set_ylim(-10,-8)
ax.set_xlabel('Log n [cm$^{-3}$]')
ax.set_ylabel('Log $f_{e^-}$')

In [ ]:
f,(ax0,ax1) = plt.subplots(2, sharex=True, figsize=(8,12))
ax0.plot(average['vanilla'].logn, average['vanilla'].logT, color='.25', label=simname['vanilla'])
for sim in simlist[1:]:
    df = average[sim]
    ax0.plot(df.logn, df.logT, label=simname[sim])
ax0.legend(loc=0)
ax0.set_ylabel('Log Temperature [K]')

ax1.plot(average['vanilla'].logn, average['vanilla'].loge, color='.25', label=simname['vanilla'])
for sim in simlist[1:]:
    df = average[sim]
    ax1.plot(df.logn, df.loge, label=simname[sim])
#ax1.legend(loc=0)
#ax1.set_xlim(8,12)
#ax1.set_ylim(-10,-8)
ax1.set_ylabel("Log free $e^-$ fraction")
ax1.set_xlabel("Log number density [cm$^{-3}$]")
f.subplots_adjust(hspace=0.02)
f.savefig('figures/phase_diagrams/binned.png', bbox_inches='tight')